In [5]:
!pip install opendatasets


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

In [7]:
dataset_path = Path("new-york-city-airbnb-open-data", "AB_NYC_2019.csv")
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data')

Dataset URL: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data


100%|██████████| 2.44M/2.44M [00:00<00:00, 4.09MB/s]


In [8]:
df = pd.read_csv(dataset_path)
df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [9]:
prices_series = df['price']
prices_series.info()
prices = prices_series.tolist()

<class 'pandas.core.series.Series'>
RangeIndex: 48895 entries, 0 to 48894
Series name: price
Non-Null Count  Dtype
--------------  -----
48895 non-null  int64
dtypes: int64(1)
memory usage: 382.1 KB


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

В датасете 48 895 строк, 16 колонок. В колонке prise 48 895 записей, пропусков, нулевых значений нет.

In [11]:
# Описательная статистика

stats = df['price'].describe()
print(stats)

count    48895.000000
mean       152.720687
std        240.154170
min          0.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64


Цена [price] - средняя цена составляет $ 152, средне-квадратичное отклонение составляет 240.



In [22]:
%%time
from numpy import mean, std

def airbnb_prices(data_frame, some_col):
    score = []
    for i in data_frame[some_col]:
        score.append(i)

    return mean(score), std(score)

print(airbnb_prices(df, "price"))

(152.7206871868289, 240.15171391941718)
CPU times: total: 31.2 ms
Wall time: 55 ms


In [25]:
pip install joblib


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [26]:
from functools import reduce

def mapper(price):
    return (price, )

def reducer(price_data1, price_data2):
        prices = []
        if len(price_data1) == 1:
            n, mean, delta_sqr = 0, 0.0, 0
            prices.append(price_data1[0])
        else:
            n, mean, delta_sqr = price_data1
        prices.append(price_data2[0])
        for price in prices:
            n += 1
            delta = price - mean
            mean += delta / n
            delta_sqr += delta * (price - mean)
        return n, mean, delta_sqr

from joblib import Parallel, delayed
import math as m

n, mean, delta_sqr = reduce(reducer, map(mapper, prices))
variance = delta_sqr / n
sdv = variance ** (1/2)
print(f'Среднее: {mean}, Дисперсия: {variance}, СКО: {sdv}')

Среднее: 152.72068718682823, Дисперсия: 57672.845698433375, СКО: 240.15171391941672


In [27]:
# Для проверки:
def mean(prices):
    return sum(prices)/len(prices)


def variance(prices):
    m = mean(prices)
    return sum([(price - m)**2 for price in prices])/len(prices)

print(f'Среднее: {mean(prices)}, Дисперсия: {variance(prices)}')

Среднее: 152.7206871868289, Дисперсия: 57672.84569843345
